<a href="https://colab.research.google.com/github/sajeewa0825/Gemma-3-fine-tuning-with-custom-dataset/blob/main/Inference_Chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import torch
model_id = "sajeewa/empathy-chat-gemma"

# **Load the Tokenizer and Model**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)


# **Step 1: Initialize Chat History**

In [ ]:
chat_history = [
    {"role": "system", "content": "You are an empathetic AI and your friend. always give lovely caring message. Understand the user's feelings.Then provide a caring response. please give response as good friend also talk with lovely word like baby, my cutey and etc. please maintian conversation word count according to the user prompt. you can use emojis for calm and try to be continue chat"}
]


# **Step 2: User Sends a Message**

In [ ]:
user_input = "I'm feeling lonely today."
chat_history.append({"role": "user", "content": user_input})


# **Step 3: Format the Chat History as Prompt**

In [ ]:
prompt = tokenizer.apply_chat_template(
    chat_history,
    tokenize=False,
    add_generation_prompt=True,
)


# **Step 4: Tokenize and Generate Response**

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Optional: stream output for live effect
from transformers import TextStreamer
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

output = model.generate(
    **inputs,
    max_new_tokens=128,
    temperature=0.7,
    top_p=0.95,
    top_k=50,
    do_sample=True,
    streamer=streamer
)

# Get the generated text
response = tokenizer.decode(output[0], skip_special_tokens=True)


# **Step 5: Add the Model’s Response to Chat History**

In [ ]:
# If streaming, you already saw output. Otherwise:
new_response = response[len(prompt):].strip()
chat_history.append({"role": "assistant", "content": new_response})

In [ ]:
chat_history

# **Step 6: Next Turn – Continue the Conversation**

In [ ]:
user_input = "Thank your support my friend. now i want to sleep, good bye and good night  "
chat_history.append({"role": "user", "content": user_input})

# **max_prompt_length**

In [ ]:
MAX_TOKENS = 2048
chat_prompt = tokenizer.apply_chat_template(chat_history, tokenize=False)
while len(tokenizer(chat_prompt).input_ids) > MAX_TOKENS:
    chat_history.pop(1)  # remove oldest user/assistant message (after system)
    chat_prompt = tokenizer.apply_chat_template(chat_history, tokenize=False)


In [ ]:
tokens = tokenizer(prompt, return_tensors="pt")
print("Token length:", tokens["input_ids"].shape[-1])
